<a href="https://colab.research.google.com/github/nurulc/ATM/blob/master/tf_tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import drive
import tensorflow_datasets as tfds
import tensorflow as tf
import os

# New Section

In [10]:

#drive.mount('content/gdrive')
# !head drive/My\ Drive/housing-ma.csv
!head "/content/drive/My Drive/Colab Notebooks/Copy of tf-tutor.ipynb"

{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"name":"Copy of tf-tutor.ipynb","provenance":[{"file_id":"1khniD3ToLJ8Cu1vfisUn--67Vz35fnqE","timestamp":1596112371136}],"collapsed_sections":[],"toc_visible":true,"mount_file_id":"1khniD3ToLJ8Cu1vfisUn--67Vz35fnqE","authorship_tag":"ABX9TyOk2NCNRpnf+tjSgVTDmwn8"},"kernelspec":{"name":"python3","display_name":"Python 3"}},"cells":[{"cell_type":"code","metadata":{"id":"ziH-agmfSefN","colab_type":"code","colab":{},"executionInfo":{"status":"ok","timestamp":1596112999836,"user_tz":240,"elapsed":2577,"user":{"displayName":"Nurul Choudhury","photoUrl":"https://lh3.googleusercontent.com/a-/AOh14Gj58tz-GdLAugW6XcFFOmCGS1h4KNC1_c7b89QMtg=s64","userId":"09061348886132893566"}}},"source":["import numpy as np\n","from google.colab import drive\n","import tensorflow_datasets as tfds\n","import tensorflow as tf\n","import os"],"execution_count":2,"outputs":[]},{"cell_type":"markdown","metadata":{"id":"9WoWUwtCSjSF","colab_type":"text"},"source":

In [ ]:
import pandas as pd
import io
data_pd = pd.read_csv(io.FileIO('sample_data/housing-ma.csv'))
data_pd


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [ ]:
dtype=np.double
def col_sort(t,col=0,descending=False):
    return t[t[:, col].sort(descending=descending)[1]]



# Normalize the data

For each column of X(data)  subtract the mean of the column 

This normalization maked the algoririthm more stable

In [ ]:
def norm(X):
    R=(np.amax(X,0)-np.amin(X,0)).reshape(-1,1)
    R[X.shape[1]-1][0] = 1.0
    M=np.mean(X,0).reshape(-1,1)
    M[X.shape[1]-1][0] = 0.0
    Mv = M.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    Sc = R.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    return [(X-Mv)/Sc,R,M]

In [ ]:
X1, Range, Mean = norm(X)
np.abs((X1*Sc+Mv - X))<1e-10

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:


# !head sample_data/housing-ma.csv
ones = np.ones
cat = np.concatenate

Z = data_pd.to_numpy(dtype=np.double,copy=True)
m = X.shape[0] # m = number of rows of data
n = X.shape[1]-1 # n = number of columns excluding the house price
X = Z[:,0:n] # remove the house price (last column)

Ones = ones(m,dtype=np.double).reshape(-1,1)
X = cat((X,Ones),1) # add a 1 to each row (to compute the intercept)
y = Z[:,n].reshape(-1,1) # get the house price
phi = np.ones(n+1,dtype=np.double).reshape(-1,1) # initial value of phi
print("initial phi\n", phi)
print("input\n", X)

print("house price (first 10)\n",y[0:10])


initial phi
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
input
 [[  0.00632  18.        2.31    ... 396.9       4.98      1.     ]
 [  0.02731   0.        7.07    ... 396.9       9.14      1.     ]
 [  0.02729   0.        7.07    ... 392.83      4.03      1.     ]
 ...
 [  0.06076   0.       11.93    ... 396.9       5.64      1.     ]
 [  0.10959   0.       11.93    ... 393.45      6.48      1.     ]
 [  0.04741   0.       11.93    ... 396.9       7.88      1.     ]]
house price (first 10)
 [[24. ]
 [21.6]
 [34.7]
 [33.4]
 [36.2]
 [28.7]
 [22.9]
 [27.1]
 [16.5]
 [18.9]]


# Numpy Multivariate regression

This method uses gradient descent to iteratively mimimize the cost function (MSE - mean standard error).

## Iterative computation of parameters _phi_ (multi variable linear regression)

$phi (\phi) $ are the regression parematers

*  find $\phi$ that minimizes cost
  *  *where:*    $Cost = MSE = \frac{(X\phi -y)^T(X\phi -y)}{2m}$
   * $\phi$ is a minimum when $\frac{\partial{Cost}}{\partial\phi} = 0$
   * i.e. $\frac{\partial{Cost}}{\partial\phi} = \frac{X^T(X\phi - y)}{m} = 0$
   * by gradient descent we can compute successive values of $\phi_j$ 
       * $  \phi_j = \phi_{j-1} - \alpha \frac{\partial{Cost}}{\partial\phi}$
       * Resulting in the following:
          * $  \phi_j = \phi_{j-1} - \alpha X^T(X\phi_{j-1} - y)$
          * $\alpha$ is the learning rate is a small scalar such as 0.05

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$

In [ ]:
import math
tr = np.transpose

def cost(X,phi,Y):
    m = X.shape[0];
    return math.sqrt((1/(2*m))*(tr(X@phi - Y)@(X@phi - Y)).item())

def npSolve(X,y,iter=2,alpha=0.05):
    W, Range,Mean = norm(X) # normalize the data

    phi = ones((X.shape[1],1),dtype=np.double)
    cost_list = []
    Y = y
    m = X.shape[0]
    
    for i in range(iter):
        delta = alpha*(1/m)*(W.T)@(W@phi - Y) 
        phi = phi - delta
        cost_val = cost(W,phi,Y)
        cost_list.append(cost_val)
        
    intercept = np.zeros_like(Mean)
    intercept[-1][0] = (phi*Mean/Range).sum().item()
    phi = phi/Range - intercept
    return [phi,cost_list]    

# Iterative computation of parameters _phi_ (multi variable linear regression)

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$


        

In [ ]:
%%time
phi,cost_list = npSolve(X,y,2000,0.5)
print(cost(X,phi,y))

3.3086923215668995
CPU times: user 53.1 ms, sys: 0 ns, total: 53.1 ms
Wall time: 56.1 ms


## Perform Multivariate regression (directly - no iteration)

We previously saw that for minimum $\phi$
* We have the following
* $\frac{X^T(X\phi - y)}{m} = 0$
  * multiplying bost sides by $m$
* $X^T(X\phi - y) = 0$
  * Multiplying by $X^T$ - _distributive property_
* $X^T(X\phi) -  X^Ty = 0$
  * Rearanging
* $(X^TX)\phi =  X^Ty$
----
   * Multiply both sides by the inverse of $X^TX$

* $(X^TX)^{-1} (X^TX)\phi = (X^TX)^{-1} X^Ty$
   * since  ${(X^TX)^{-1} (X^TX)} = I (identity)$
* we get the following: $I \phi = (X^TX)^{-1} X^Ty$
----
 **Therefore we get the very simple formula for $\phi$**
   * *$\phi = (X^TX)^{-1} X^Ty$*
----
&nbsp;

Note the following rules for matrix multiply
* $A(BC) = (AB)C$
  * and
* $A(B+C) = AB + BC$ - _distributive property_



In [ ]:
%%time

inv = np.linalg.inv # invert a matrix

# See how simple the formulation can be using only linear algebra
def solve(X,y):
  return inv(X.T@X) @ (X.T@y)

phi2 = solve(X,y)  
print("cost=", cost(X,phi2,y),"diff = ", cost(X,phi,y)-cost(X,phi2,y)) # compare the cost of direct method and iterative method
#[phi,phi2, cost(X,phi2,y)-cost(X,phi,y), np.abs((phi-phi2)/phi*100)]

cost= 3.308687895656616 diff =  4.425910283600842e-06
CPU times: user 1.46 ms, sys: 895 µs, total: 2.36 ms
Wall time: 1.57 ms


## Ignore the rest of this


In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

v1 = data.take(1)
v1

<TakeDataset shapes: OrderedDict([(CRIM, (5,)), (ZN, (5,)), (INDUS, (5,)), (CHAS, (5,)), (NOX, (5,)), (RM, (5,)), (AGE, (5,)), (DIS, (5,)), (RAD, (5,)), (TAX, (5,)), (PTRATIO, (5,)), (B, (5,)), (LSTAT, (5,)), (MEDV, (5,))]), types: OrderedDict([(CRIM, tf.float32), (ZN, tf.float32), (INDUS, tf.float32), (CHAS, tf.int32), (NOX, tf.float32), (RM, tf.float32), (AGE, tf.float32), (DIS, tf.float32), (RAD, tf.int32), (TAX, tf.float32), (PTRATIO, tf.float32), (B, tf.float32), (LSTAT, tf.float32), (MEDV, tf.float32)])>

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=6, suppress=True)

In [ ]:
!head {train_file_path}
LABEL_COLUMN = 'survived'
LABELS = [0, 1]
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [ ]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [29. 28.  2. 31. 18.]
n_siblings_spouses  : [1 0 4 1 0]
parch               : [0 1 1 1 0]
fare                : [27.721 33.    29.125 26.25  11.5  ]
class               : [b'Second' b'Second' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'y']


# New Section